In [3]:
#for windows users:
#pip install bitermplus

#for mac users:
# xcode-select --install
# brew install libomp
# pip3 install bitermplus

# pip install tmplot
# pip install tomotopy

In [1]:
import bitermplus as btm
import numpy as np
import pandas as pd

import json 
import csv
import nltk
import unicodedata
import re
import sys
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim, spacy
from gensim.models.ldamulticore import LdaMulticore
import re
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [2]:
df = pd.read_csv("../pos_neg_india.csv")
df

,Unnamed: 0,index,Text,new_text,languages,translated,new_languages,pos_tagged_words,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,0,0,soniya mummy hoti toh covid me magic stick ghu...,soniya mummy hoti toh covid me magic stick ghu...,Unknown,soniya mummy hoti toh covid me magic stick ghu...,Unknown,"[('soniya', 'JJ'), ('mummy', 'NN'), ('hot', 'J...","['soniya', 'mummy', 'hot', 'toh', 'couid', 'ma...",['death'],['death'],subjective,0
1,12,12,is covid ki waja se buht saray logon ne apne p...,is covid ki waja se buht saray logon ne apne p...,Unknown,is covid ki waja se buht saray logon ne apne p...,Unknown,"[('is', 'VBZ'), ('couid', 'JJ'), ('ki', 'NN'),...","['couid', 'ki', 'raja', 'se', 'spray', 'logon'...","['hair', 'ply', 'vaccination']","['hair', 'ply', 'vaccination']",subjective,2
2,18,18,deaths due covid what is vaccination rate what...,deaths due covid what is vaccination rate what...,"ENGLISH, Unknown",deaths due covid what is vaccination rate what...,"ENGLISH, Unknown","[('deaths', 'NNS'), ('due', 'JJ'), ('couid', '...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2
3,28,28,vaccination against covid- 19 is necessity tod...,vaccination against covid- 19 is necessity tod...,ENGLISH,vaccination against covid- 19 is necessity tod...,ENGLISH,"[('vaccination', 'NN'), ('against', 'IN'), ('c...","['vaccination', 'couid', 'nineteen', 'necessit...","['necessity', 'today', 'scenario', 'hope', 'pr...","['necessity', 'today', 'scenario', 'problem']",subjective,2
4,33,33,very surprising that unvaccinated population &...,very surprising that unvaccinated population &...,ENGLISH,very surprising that unvaccinated population &...,ENGLISH,"[('very', 'RB'), ('surprising', 'JJ'), ('that'...","['surprising', 'vaccinated', 'population', 'yr...","['surprising', 'vaccinated', 'population', 'yr...","['population', 'yrs', 'offline', 'class', 'amp...",subjective,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37618,95179,95179,nature seems have already started working savi...,nature seems have already started working savi...,ENGLISH,nature seems have already started working savi...,ENGLISH,"[('nature', 'NN'), ('seems', 'VBZ'), ('have', ...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'generation', 'vaccinesaveslive', '...",subjective,2
37619,95180,95180,"dear , we #india 🇮🇳has helped this #world incl...","dear , we #india Indiahas helped this #world i...",ENGLISH,"dear , we #india Indiahas helped this #world i...",ENGLISH,"[('dear', 'IN'), ('we', 'PRP'), ('india', 'VBP...","['dear', 'india', 'indians', 'help', 'world', ...","['dear', 'help', 'world', 'include', 'kindness...","['world', 'kindnessmatter', 'vaccinesaveslive']",subjective,2
37620,95184,95184,our sincere request state central govt 🙏🙏. ple...,our sincere request state central govt folded_...,ENGLISH,our sincere request state central govt folded_...,ENGLISH,"[('our', 'PRP$'), ('sincere', 'JJ'), ('request...","['sincere', 'request', 'state', 'central', 'go...","['sincere', 'request', 'state', 'central', 'go...","['request', 'state', 'govt', 'share', 'vaccine...",subjective,2
37621,95185,95185,thanks your appreciation valuable feedback abo...,thanks your appreciation valuable feedback abo...,ENGLISH,thanks your appreciation valuable feedback abo...,ENGLISH,"[('thanks', 'NNS'), ('your', 'PRP$'), ('apprec...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'feedback', 'vaccine...",subjective,2


In [3]:
df.drop(columns = ['Unnamed: 0', 'Text', 'index', 'new_text', 'languages', 'new_languages', 'pos_tagged_words'], inplace = True)
df

,translated,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,soniya mummy hoti toh covid me magic stick ghu...,"['soniya', 'mummy', 'hot', 'toh', 'couid', 'ma...",['death'],['death'],subjective,0
1,is covid ki waja se buht saray logon ne apne p...,"['couid', 'ki', 'raja', 'se', 'spray', 'logon'...","['hair', 'ply', 'vaccination']","['hair', 'ply', 'vaccination']",subjective,2
2,deaths due covid what is vaccination rate what...,"['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2
3,vaccination against covid- 19 is necessity tod...,"['vaccination', 'couid', 'nineteen', 'necessit...","['necessity', 'today', 'scenario', 'hope', 'pr...","['necessity', 'today', 'scenario', 'problem']",subjective,2
4,very surprising that unvaccinated population &...,"['surprising', 'vaccinated', 'population', 'yr...","['surprising', 'vaccinated', 'population', 'yr...","['population', 'yrs', 'offline', 'class', 'amp...",subjective,0
...,...,...,...,...,...,...
37618,nature seems have already started working savi...,"['nature', 'seem', 'already', 'start', 'work',...","['nature', 'seem', 'already', 'start', 'work',...","['nature', 'generation', 'vaccinesaveslive', '...",subjective,2
37619,"dear , we #india Indiahas helped this #world i...","['dear', 'india', 'indians', 'help', 'world', ...","['dear', 'help', 'world', 'include', 'kindness...","['world', 'kindnessmatter', 'vaccinesaveslive']",subjective,2
37620,our sincere request state central govt folded_...,"['sincere', 'request', 'state', 'central', 'go...","['sincere', 'request', 'state', 'central', 'go...","['request', 'state', 'govt', 'share', 'vaccine...",subjective,2
37621,thanks your appreciation valuable feedback abo...,"['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'feedback', 'vaccine...",subjective,2


In [4]:
df['words_processed_noun'].values[0][0]

'['

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [6]:
tokens_words_processed_noun = list(sent_to_words(df['words_processed_noun']))

In [7]:
from nltk.corpus import stopwords
stop_list = stopwords.words('english')
high_freq_known_words = ['covid','couid', 'pfizer', 'amp', 'people', 'covidvaccine', 'covishield', 'moderna', 'vaccine', 'astrazeneca', 'vaccinate', 'vaccinated', 'vaccination', 'vaccines']

stop_list.extend(high_freq_known_words)

test_words_only1 = ['not', 'vaccine', 'covid', 'and', 'vegan', 'cruelty', 'free', 'i', 'use', 'elf', 'cosmetics', 'every', 'day', 'and', 'i', 'love', 'it']
test_stopremoved = [w for w in test_words_only1 if w not in stop_list or w == 'not']
print(test_stopremoved)

['not', 'vegan', 'cruelty', 'free', 'use', 'elf', 'cosmetics', 'every', 'day', 'love']


In [8]:
stop_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [9]:
df['words_processed_noun'] = tokens_words_processed_noun

In [11]:
# remove stop words from tokenised_words_only

tokenised_stopremoved = [[w for w in each if w not in stop_list or w == 'not'] for each in tokens_words_processed_noun]
tokenised_stopremoved

[['death'],
 ['hair', 'ply'],
 ['death', 'rate', 'harm', 'harm'],
 ['necessity', 'today', 'scenario', 'problem'],
 ['population',
  'yrs',
  'offline',
  'class',
  'examination',
  'situation',
  'safety',
  'class',
  'examination'],
 ['way', 'thing', 'millennium', 'peril', 'job', 'process', 'retweet'],
 ['symptom', 'example', 'country', 'number', 'death', 'population', 'life'],
 ['follower', 'pledge', 'service', 'humanity'],
 ['case', 'case', 'testing', 'taboo'],
 ['intellectual',
  'education',
  'leader',
  'health',
  'system',
  'pandemic',
  'nation',
  'service'],
 ['agreement', 'student', 'travel', 'guideline'],
 ['system',
  'conversion',
  'appeasement',
  'health',
  'service',
  'infection',
  'rate',
  'death',
  'rate',
  'rate',
  'welfare',
  'isis'],
 ['oppression',
  'yesterday',
  'appointment',
  'wife',
  'today',
  'worker',
  'present',
  'time',
  'money'],
 ['effectiveness',
  'life',
  'number',
  'rajasthan',
  'safetyfirst',
  'thursdayvibe'],
 ['campaign'

In [12]:
df['words_processed_noun'].values[0][0]

'death'

In [13]:
df['tokenised_stopremoved'] = tokenised_stopremoved

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37623 entries, 0 to 37622
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   translated                         37623 non-null  object
 1   words_processed_all                37623 non-null  object
 2   words_processed_noun_adj_verb_adv  37623 non-null  object
 3   words_processed_noun               37623 non-null  object
 4   text_type                          37623 non-null  object
 5   sentiment                          37623 non-null  int64 
 6   tokenised_stopremoved              37623 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.0+ MB


In [15]:
df['sentiment'].value_counts()

0    20029
2    17594
Name: sentiment, dtype: int64

In [16]:
positive_df = df[df['sentiment'] == 2]

In [17]:
positive_df

,translated,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment,tokenised_stopremoved
1,is covid ki waja se buht saray logon ne apne p...,"['couid', 'ki', 'raja', 'se', 'spray', 'logon'...","['hair', 'ply', 'vaccination']","[hair, ply, vaccination]",subjective,2,"[hair, ply]"
2,deaths due covid what is vaccination rate what...,"['death', 'due', 'couid', 'vaccination', 'rate...","['death', 'due', 'couid', 'vaccination', 'rate...","[death, couid, vaccination, rate, harm, harm]",subjective,2,"[death, rate, harm, harm]"
3,vaccination against covid- 19 is necessity tod...,"['vaccination', 'couid', 'nineteen', 'necessit...","['necessity', 'today', 'scenario', 'hope', 'pr...","[necessity, today, scenario, problem]",subjective,2,"[necessity, today, scenario, problem]"
5,dr. deserved #1 minister's award his exception...,"['deserved', 'one', 'minister', 'award', 'exce...","['deserved', 'way', 'manage', 'thing', 'millen...","[way, thing, millennium, peril, couid, job, va...",subjective,2,"[way, thing, millennium, peril, job, process, ..."
6,"yeah i still do not get why, symptoms are much...","['yeah', 'still', 'not', 'get', 'symptom', 'mu...","['still', 'symptom', 'much', 'less', 'severe',...","[symptom, vaccination, example, country, numbe...",subjective,2,"[symptom, example, country, number, death, pop..."
...,...,...,...,...,...,...,...
37618,nature seems have already started working savi...,"['nature', 'seem', 'already', 'start', 'work',...","['nature', 'seem', 'already', 'start', 'work',...","[nature, generation, life, strengthen]",subjective,2,"[nature, generation, life, strengthen]"
37619,"dear , we #india Indiahas helped this #world i...","['dear', 'india', 'indians', 'help', 'world', ...","['dear', 'help', 'world', 'include', 'kindness...","[world, kindnessmatter]",subjective,2,"[world, kindnessmatter]"
37620,our sincere request state central govt folded_...,"['sincere', 'request', 'state', 'central', 'go...","['sincere', 'request', 'state', 'central', 'go...","[request, state, govt, share]",subjective,2,"[request, state, govt, share]"
37621,thanks your appreciation valuable feedback abo...,"['thank', 'appreciation', 'valuable', 'feedbac...","['thank', 'appreciation', 'valuable', 'feedbac...","[thank, appreciation, feedback]",subjective,2,"[thank, appreciation, feedback]"


In [18]:
texts = positive_df['words_processed_noun']

In [19]:
new_text_noun = []

for row in texts:
    # print(type(row))
    new_row = " ".join(row)
    new_text_noun.append(new_row)


In [20]:
new_text_noun

['hair ply vaccination',
 'death couid vaccination rate harm harm',
 'necessity today scenario problem',
 'way thing millennium peril couid job vaccination process retweet',
 'symptom vaccination example country number death population couid life',
 'agreement couid student travel guideline',
 'campaign',
 'case malaysia vaccination booster',
 'age fact vaccinate child parent',
 'wife couid adult ease people social_distance mask vaccination',
 'year today battle couid vaccination drive virus threat heel',
 'mess subsidy program crore citizen food growth rate percentage couid digitization benefit transfer',
 'world country history leave vaccination couid part vote',
 'couid vaccine immunization test need avoid vaccination',
 'today nurse couid norm people child count amp vaccination detail',
 'wolf shielding vaccination couid',
 'vaccine people couid litter day people dose vaccination decrease number case cat',
 'team roper report arrival vaccination certificate',
 'village guitar couid

In [21]:
positive_df['new_text_noun'] = new_text_noun

/var/folders/lg/w8d05wzd48q3m5_s46qt2fxc0000gn/T/ipykernel_86643/1604708766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_df['new_text_noun'] = new_text_noun


In [22]:
texts = positive_df['new_text_noun']

In [23]:
texts

1                                     hair ply vaccination
2                   death couid vaccination rate harm harm
3                         necessity today scenario problem
5        way thing millennium peril couid job vaccinati...
6        symptom vaccination example country number dea...
                               ...                        
37618                    nature generation life strengthen
37619                                 world kindnessmatter
37620                             request state govt share
37621                          thank appreciation feedback
37622        dose covidvaccine amp amp follow vaccine folk
Name: new_text_noun, Length: 17594, dtype: object

In [24]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

high_freq_known_words = ['covid','couid', 'pfizer', 'amp', 'people', 'covidvaccine', 'covishield', 'moderna', 'vaccine', 'astrazeneca', 'vaccinate', 'vaccinated', 'vaccination', 'vaccines']

stop_words.extend(high_freq_known_words)

In [25]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=4, M=20, alpha=50/4, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [00:12<00:00, 49.60it/s]


In [26]:
stable_topics = [0, 1, 2, 3]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [27]:
top_words

,topic0,topic1,topic2,topic3
0,student,country,mask,thank
1,school,day,family,today
2,child,time,virus,dose
3,country,world,day,staff
4,government,population,time,center
5,staff,government,dose,day
6,life,year,today,time
7,time,case,home,family
8,state,state,friend,hospital
9,price,virus,corona,team


## 5 topics

In [28]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=5, M=20, alpha=50/5, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [00:11<00:00, 53.44it/s]


In [29]:
stable_topics = [0, 1, 2, 3, 4]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [30]:
top_words

,topic0,topic1,topic2,topic3,topic4
0,student,country,virus,country,today
1,school,world,mask,state,thank
2,day,thank,day,time,dose
3,child,today,time,day,staff
4,travel,time,dose,government,center
5,time,health,case,world,day
6,home,day,family,help,time
7,family,year,symptom,population,family
8,country,government,corona,price,hospital
9,quarantine,work,today,govt,team


## 6 topics

In [31]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=6, M=20, alpha=50/6, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [00:10<00:00, 57.41it/s]


In [32]:
stable_topics = [0, 1, 2, 3, 4, 5]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [33]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5
0,day,country,virus,today,number,student
1,injection,world,mask,thank,day,staff
2,symptom,time,time,dose,time,day
3,time,government,case,staff,post,school
4,friend,day,day,center,love,time
5,family,state,family,day,news,country
6,fever,nation,dose,time,certificate,travel
7,pain,population,corona,family,thing,test
8,needle,health,hand,hospital,team,government
9,year,citizen,distance,team,radar,home


## 7 topics

In [34]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=7, M=20, alpha=50/7, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [00:14<00:00, 41.55it/s]


In [35]:
stable_topics = [0, 1, 2, 3, 4, 5, 6]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [36]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6
0,student,country,day,day,mask,thank,day
1,school,world,time,family,virus,today,travel
2,child,time,case,love,corona,dose,staff
3,kid,government,dose,time,today,staff,time
4,year,state,virus,friend,family,center,quarantine
5,life,nation,country,injection,hand,day,country
6,home,health,year,today,distance,time,test
7,exam,thank,number,symptom,time,family,case
8,time,citizen,population,hour,dose,hospital,person
9,family,work,person,needle,safety,slot,home


## 8 topics

In [37]:
# TRY DIFFERENT NUM TOPICS
# keep beta = 0.09, try different beta if results arent great
# iterations = 600 seems decent

X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
biterms = btm.get_biterms(docs_vec)

# Initializing and running model
model = btm.BTM(
    X, vocabulary, seed=12321, T=8, M=20, alpha=50/8, beta=0.09)
model.fit(biterms, iterations=600)

100%|██████████| 600/600 [00:14<00:00, 42.65it/s]


In [38]:
stable_topics = [0, 1, 2, 3, 4, 5, 6, 7]
top_words = btm.get_top_topic_words(
    model,
    words_num=12,
    topics_idx=stable_topics)

In [39]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7
0,student,price,country,day,today,mask,staff,day
1,child,government,world,symptom,thank,virus,center,travel
2,school,company,time,case,dose,corona,hospital,quarantine
3,year,state,day,dose,family,distance,help,country
4,kid,production,health,time,day,hand,time,time
5,life,life,government,week,time,time,day,test
6,exam,cost,population,injection,friend,follow,dose,home
7,parent,country,nation,death,team,safety,thank,family
8,teacher,world,thank,person,staff,family,certificate,traveler
9,time,money,citizen,patient,parent,dose,government,case
